In [2]:
import pandas as pd
import re

# Read the list of filenames from the configuration file
with open('../file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'

# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = f"../data/{file}"
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])


In [3]:

data['date'] = pd.to_datetime(data['date'])

data = data[data['date'].dt.year == 2025]



In [4]:
data["user"] = data["user"].replace("Banties1g", "banties_x")
data["user"] = data["user"].replace("banties1g", "banties_x")
data["user"] = data["user"].replace("chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("CHILI_POE", "chili_con_bacon")
data["user"] = data["user"].replace("chili_conbacon", "chili_con_bacon")
data["user"] = data["user"].replace("Wirelesss_", "W1r3lesss")
data["user"] = data["user"].replace("treklul", "trek44_")
data["user"] = data["user"].replace("ttrek_", "trek44_")
data["user"] = data["user"].replace("trek_x", "trek44_")
data["user"] = data["user"].replace("TriplesingleJ", "TripleSingleJames")
data["user"] = data["user"].replace("uuccugr", "uwu_cougar")
data["user"] = data["user"].replace("uuccugr", "uuccugr_")
data["user"] = data["user"].replace("StanIV4_", "stan_iv4")

In [5]:
from collections import defaultdict

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}

In [6]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [7]:
# Count the number of messages per user
message_counts = data.groupby("user")["message"].count()

# Filter users with 25 or more messages
users_with_25_or_more = message_counts[message_counts >= 25].index

# Filter the original DataFrame to keep only these users
data = data[data["user"].isin(users_with_25_or_more)]

In [8]:
from collections import defaultdict

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}

In [9]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [10]:
# Convert date to datetime format
data["date"] = pd.to_datetime(data["date"])



In [11]:
# Truncate datetime to just the day (removing time)
data["day"] = data["date"].dt.date  # Extract only the date part



In [12]:
# Group by 'day' and 'user' and calculate the message count per day per user
data["message_count"] = 1  # Assign 1 for each message to count them
daily_counts = data.groupby(["day", "user"])["message_count"].count().reset_index()



In [13]:
# Pivot the table to create a user-wise table for each day
pivot_table = daily_counts.pivot(index="day", columns="user", values="message_count").fillna(0)


In [14]:

# Add a cumulative sum for each user across the days
cumulative_pivot = pivot_table.cumsum()


In [15]:

# Print the result
print(cumulative_pivot)

user        00cad  00skysea00  00yopop  010justwatch  01gomesss  06RoB  \
day                                                                      
2025-01-01    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-03    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-04    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-05    0.0         0.0      0.0           0.0        0.0    0.0   
2025-01-06    0.0         0.0      0.0           0.0        0.0    0.0   
...           ...         ...      ...           ...        ...    ...   
2025-11-19   55.0        26.0     55.0          48.0       47.0  304.0   
2025-11-20   55.0        26.0     55.0          48.0       47.0  304.0   
2025-11-21   55.0        26.0     55.0          48.0       47.0  304.0   
2025-11-22   55.0        26.0     55.0          48.0       47.0  304.0   
2025-11-23   55.0        26.0     55.0          48.0       47.0  304.0   

user        07moki  0SHADOWIX  0_0and

In [16]:
pivot_data_cleaned_transposed = cumulative_pivot.T
pivot_data_cleaned_transposed = cumulative_pivot.T.reset_index()

In [17]:
pivot_data_cleaned_transposed.head(5)
pivot_data_cleaned_transposed.tail(5)

day,user,2025-01-01,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-08,2025-01-09,2025-01-10,2025-01-11,...,2025-11-13,2025-11-14,2025-11-15,2025-11-16,2025-11-17,2025-11-19,2025-11-20,2025-11-21,2025-11-22,2025-11-23
6375,餅藍,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0,174.0
6376,기분조앙,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0
6377,너의이탈리아아저씨,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0,31.0
6378,안톤958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0
6379,알래스카해달,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2828.0,2924.0,3011.0,3011.0,3082.0,3167.0,3167.0,3167.0,3206.0,3279.0


In [22]:
# --- Step 1 & 2: Calculate rank and identify top 20 users ---

# Calculate rank for each date (column) in descending order.
# The user with the highest cumulative sum gets rank 1.
ranked_df = pivot_data_cleaned_transposed.rank(axis=0, ascending=False, method='min')

# Identify users who were in the top 20 on at least one date.
# This creates a boolean Series where True means the user was in the top 20 at least once.
users_in_top_15_at_least_once = (ranked_df <= 20).any(axis=1)

# Get the list of users (their index labels) who meet the criteria.
users_to_keep = users_in_top_15_at_least_once[users_in_top_15_at_least_once].index

# --- Step 3: Filter the DataFrame ---

# Create a new DataFrame containing only the users who were in the top 20 at least once.
filtered_users_df = pivot_data_cleaned_transposed.loc[users_to_keep]

print("Original DataFrame shape:", pivot_data_cleaned_transposed.shape)
print("Filtered DataFrame shape:", filtered_users_df.shape)

Original DataFrame shape: (6380, 279)
Filtered DataFrame shape: (67, 279)


In [23]:
filtered_users_df.head()

day,user,2025-01-01,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-08,2025-01-09,2025-01-10,2025-01-11,...,2025-11-13,2025-11-14,2025-11-15,2025-11-16,2025-11-17,2025-11-19,2025-11-20,2025-11-21,2025-11-22,2025-11-23
28,1206paul_,51.0,109.0,172.0,238.0,309.0,415.0,468.0,516.0,610.0,...,6497.0,6509.0,6511.0,6550.0,6555.0,6576.0,6580.0,6612.0,6632.0,6659.0
201,Aloddin,0.0,114.0,124.0,146.0,173.0,191.0,230.0,237.0,248.0,...,4329.0,4329.0,4329.0,4331.0,4331.0,4331.0,4331.0,4333.0,4333.0,4333.0
204,Aluminiumminimumimmunity,39.0,218.0,251.0,334.0,368.0,389.0,413.0,426.0,474.0,...,12136.0,12156.0,12184.0,12214.0,12269.0,12326.0,12357.0,12365.0,12415.0,12455.0
290,BenXBari,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0
418,CrazeE420xd,299.0,467.0,504.0,817.0,995.0,1337.0,1550.0,1795.0,1827.0,...,23367.0,23367.0,23367.0,23373.0,23373.0,23373.0,23373.0,23373.0,23373.0,23373.0


In [24]:
filtered_users_df.info

<bound method DataFrame.info of day                       user  2025-01-01  2025-01-03  2025-01-04  \
28                   1206paul_        51.0       109.0       172.0   
201                    Aloddin         0.0       114.0       124.0   
204   Aluminiumminimumimmunity        39.0       218.0       251.0   
290                   BenXBari         0.0         0.0         0.0   
418                CrazeE420xd       299.0       467.0       504.0   
...                        ...         ...         ...         ...   
6375                        餅藍         0.0         0.0         0.0   
6376                      기분조앙         0.0         0.0         0.0   
6377                 너의이탈리아아저씨         0.0         0.0         0.0   
6378                     안톤958         0.0         0.0         0.0   
6379                    알래스카해달         0.0         0.0         0.0   

day   2025-01-05  2025-01-06  2025-01-08  2025-01-09  2025-01-10  2025-01-11  \
28         238.0       309.0       415.0       

In [25]:
filtered_users_df.head()

day,user,2025-01-01,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-08,2025-01-09,2025-01-10,2025-01-11,...,2025-11-13,2025-11-14,2025-11-15,2025-11-16,2025-11-17,2025-11-19,2025-11-20,2025-11-21,2025-11-22,2025-11-23
28,1206paul_,51.0,109.0,172.0,238.0,309.0,415.0,468.0,516.0,610.0,...,6497.0,6509.0,6511.0,6550.0,6555.0,6576.0,6580.0,6612.0,6632.0,6659.0
201,Aloddin,0.0,114.0,124.0,146.0,173.0,191.0,230.0,237.0,248.0,...,4329.0,4329.0,4329.0,4331.0,4331.0,4331.0,4331.0,4333.0,4333.0,4333.0
204,Aluminiumminimumimmunity,39.0,218.0,251.0,334.0,368.0,389.0,413.0,426.0,474.0,...,12136.0,12156.0,12184.0,12214.0,12269.0,12326.0,12357.0,12365.0,12415.0,12455.0
290,BenXBari,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0,45778.0
418,CrazeE420xd,299.0,467.0,504.0,817.0,995.0,1337.0,1550.0,1795.0,1827.0,...,23367.0,23367.0,23367.0,23373.0,23373.0,23373.0,23373.0,23373.0,23373.0,23373.0


In [26]:
filtered_users_df.to_excel('chattersRace2025.xlsx', sheet_name='Pivot Table')

In [ ]:
data.shape

(1980156, 6)